In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import threading
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Using Chrome browser
driver = webdriver.Chrome()

In [3]:
url = 'https://books.toscrape.com/'
driver.get(url)  

In [22]:
#hàm lấy link sách trong trang
def get_link(link):
    driver.get(link)
    sach=driver.find_elements(By.CLASS_NAME, value='col-xs-6')
    for s in sach:
        links.append(s.find_element(By.XPATH, value='.//a').get_attribute('href'))

In [ ]:
get_link('https://books.toscrape.com/catalogue/page-1.html')
print(len(links))

20


In [9]:
data={
    'Tên Sách':[],
    'Giá':[],
    'Tình trạng':[],
    'Số lượng':[],
    'Đánh Giá':[]
}

In [28]:
#hàm lấy thông tin chi tiết từng sách
def get_in():
    ten=driver.find_element(By.XPATH, value='//*[@id="content_inner"]/article/div[1]/div[2]/h1').text
    gia=driver.find_element(By.XPATH, value='//*[@id="content_inner"]/article/div[1]/div[2]/p[1]').text
    gia_sau=gia.replace("£", "")
    tinh_trang=driver.find_element(By.XPATH, value='//*[@id="content_inner"]/article/div[1]/div[2]/p[2]').text
    if 'In stock' in tinh_trang:
        tt='Còn hàng'
        so_luong=tinh_trang.split("(")[1].split(" available)")[0]
    else:
        tt='Hết hàng'
        so_luong=None
    danh_gia=driver.find_element(By.XPATH, value='//*[@id="content_inner"]/article/div[1]/div[2]/p[3]').get_attribute('class').split(" ")[1]
    if danh_gia=='One':
        so_sao=1
    if danh_gia=='Two':
        so_sao=2
    if danh_gia=='Three':
        so_sao=3
    if danh_gia=='Four':
        so_sao=4
    if danh_gia=='Five':
        so_sao=5
    data['Tên Sách'].append(ten)
    data['Giá'].append(gia_sau)
    data['Tình trạng'].append(tt)
    data['Số lượng'].append(so_luong)
    data['Đánh Giá'].append(so_sao)
get_in()

In [29]:
n=3
links=[]
for i in range(1, n+1):
    url=f'https://books.toscrape.com/catalogue/page-{i}.html'
    get_link(url)
    print(url)
    i+=1
print(f'lấy được {len(links)} sách')
for link in links:
    driver.get(link)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.content")))
    get_in()

https://books.toscrape.com/catalogue/page-1.html
https://books.toscrape.com/catalogue/page-2.html
https://books.toscrape.com/catalogue/page-3.html
lấy được 60 sách


In [33]:
df=pd.DataFrame(data)
df.to_excel('sach.xlsx')